## Implementing binary decision trees

The goal of this notebook is to implement your own binary decision tree classifier. You will:
    
* Use SFrames to do some feature engineering.
* Transform categorical variables into binary variables.
* Write a function to compute the number of misclassified examples in an intermediate node.
* Write a function to find the best feature to split on.
* Build a binary decision tree from scratch.
* Make predictions using the decision tree.
* Evaluate the accuracy of the decision tree.
* Visualize the decision at the root node.

**Important Note**: In this assignment, we will focus on building decision trees where the data contain **only binary (0 or 1) features**. This allows us to avoid dealing with:
* Multiple intermediate nodes in a split
* The thresholding issues of real-valued features.

This assignment **may be challenging**, so brace yourself :)

# Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [112]:
import turicreate
import numpy as np

# Load the lending club dataset

We will be using the same [LendingClub](https://www.lendingclub.com/) dataset as in the previous assignment.

In [113]:
loans = turicreate.SFrame('../data/lending-club-data.sframe/')

Like the previous assignment, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [114]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if x == 0 else -1)
loans = loans.remove_column('bad_loans')

Unlike the previous assignment where we used several features, in this assignment, we will just be using 4 categorical
features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.

In [115]:
features = ['grade',  # grade of the loan
            # 'sub_grade',
            'term',  # the term of the loan
            'home_ownership',  # home_ownership status: own, mortgage or rent
            'short_emp', #short duration years of employment
            # 'emp_length',  # number of years of employment
            'is_inc_v' #verified indentity
            ]


target = 'safe_loans'
loans = loans[features + [target]]

Let's explore what the dataset looks like.

In [116]:
loans

grade,term,home_ownership,short_emp,is_inc_v,safe_loans
B,36 months,RENT,0,Verified,1
C,60 months,RENT,1,Source Verified,-1
C,36 months,RENT,0,Not Verified,1
C,36 months,RENT,0,Source Verified,1
A,36 months,RENT,0,Source Verified,1
E,36 months,RENT,0,Source Verified,1
F,60 months,OWN,0,Source Verified,-1
B,60 months,RENT,1,Verified,-1
C,60 months,OWN,0,Not Verified,1
B,36 months,OWN,0,Source Verified,1


## Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use `seed=1` so everyone gets the same results.

In [117]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw) / float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed=1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

Percentage of safe loans                 : 0.5022361744216048
Percentage of risky loans                : 0.4977638255783951
Total number of loans in our new dataset : 46508


In [118]:
risky_loans

grade,term,home_ownership,short_emp,is_inc_v,safe_loans
C,60 months,RENT,1,Source Verified,-1
F,60 months,OWN,0,Source Verified,-1
B,60 months,RENT,1,Verified,-1
C,36 months,RENT,1,Source Verified,-1
B,36 months,RENT,0,Source Verified,-1
B,36 months,RENT,0,Verified,-1
B,36 months,RENT,0,Not Verified,-1
C,36 months,RENT,0,Not Verified,-1
D,60 months,RENT,0,Not Verified,-1
A,36 months,MORTGAGE,0,Source Verified,-1


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in "[Learning from Imbalanced Data](http://www.ele.uri.edu/faculty/he/PDFfiles/ImbalancedLearning.pdf)" by Haibo He and Edwardo A. Garcia, *IEEE Transactions on Knowledge and Data Engineering* **21**(9) (June 26, 2009), p. 1263–1284. For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Transform categorical data into binary features

In this assignment, we will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and Turi Create tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.

In [119]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)

    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data = loans_data.remove_column(feature)
    loans_data = loans_data.add_columns(loans_data_unpacked)

Let's see what the feature columns look like now:

In [120]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable


In [121]:
print("Number of features (after binarizing categorical variables) = %s" % len(features))

Number of features (after binarizing categorical variables) = 18


## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [122]:
train_data, test_data = loans_data.random_split(.8, seed=1)


# Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

## Function to count number of mistakes while predicting majority class

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.


** Steps to follow **:
* **Step 1:** Calculate the number of safe loans and risky loans.

* **Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.

* **Step 3:** Return the number of **mistakes**.


In [123]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0

    # Count the number of 1's (safe loans)
    sl_count = len([value for value in labels_in_node if value == 1])

    # Count the number of -1's (risky loans)
    rl_count = len([value for value in labels_in_node if value == -1])

    # Return the number of mistakes that the majority classifier makes.
    return min(sl_count, rl_count)



## Function to calculate the entropy
Now, we will write a function that calculates the **entropy** point. This will be used to help determine which feature is the best to split on at a given node of the tree.


** Steps to follow **:
* **Step 1:** Calculate the number of safe loans and risky loans.

* **Step 2:** Formula: 
$$
{entropy} = -{leftPercent}*{log2(leftPercent)} - {rightPercent}*{log2(rightPercent)}
$$

* **Step 3:** Return the number of **entropy**.

In [124]:
def calculateEntropy(target_set):

    if len(target_set) == 0:
        return 0

    #this is the target base on feature 
    good_loans = len([value for value in target_set if value == 1])

    bad_loans = len([value for value in target_set if value == -1])


    left = bad_loans / float(len(target_set))
    right = good_loans / float(len(target_set))

    if left == right: return 1

    entropyLeft = -(left * np.log2(left)) if left != 0 else 0
    entropyRight = -(right * np.log2(right)) if right != 0 else 0

    return entropyLeft + entropyRight

## Function to calculate gini index 
Now, we will write a function that calculates the **Gini index** point. You can understand the Gini index as a cost function used to evaluate splits in the dataset.


** Steps to follow **:
* **Step 1:** Calculate the number of safe loans and risky loans.

* **Step 2:** Formula: 
$$
{giniIndex} = 1 - (leftPercent^2 + rightPercent^2)
$$

* **Step 3:** Return the number of **Gini index**.

In [125]:
def calculateGiniIndex(target_set):

    if len(target_set) == 0:
        return 0

    #this is the target base on feature 
    good_loans = len([value for value in target_set if value == 1])

    bad_loans = len([value for value in target_set if value == -1])

    left = bad_loans / float(len(target_set))
    right = good_loans / float(len(target_set))


    if left == right: return 0.5

    return 1 - (left**2 + right**2) 

## Function to pick best feature to split on

**Follow these steps:**
* **Step 1:** Loop over each feature in the feature list

* **Step 2:** Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). Make sure the **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.

* **Step 3:** Calculate the number of points in both groups of data and use the above formula to compute the final percentage points

* **Step 4:** If the computed percentage points is smaller than the best points found so far, store this **feature and its points**.


**Classification Error Criteria** 
The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the **classification error** is defined as follows:
$$
{classificationError} = \frac{{ mistakes}}{{totalExamples}}
$$

**Entropy & Gini Criteria**
The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the entropy of each split and return the feature that had the smallest points when split on.

Recall that the **average entropy** is defined as follows:
$$
{averageEntropy} = {leftPercent}*{entropy(left)} + {rightPercent}*{entropy(right)}
$$

In [126]:
def best_splitting_feature(data, features, target, criteria = "error"):
    best_feature = None  # Keep track of the best feature

    # Note: Since percent is always <= 1, we should intialize it with something larger than 1.
    best_percentage_points = 2  # Keep track of the best percent points so far

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))

    # Loop through each feature to consider splitting on that feature
    for feature in features:

        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]

        # The right split will have all data points where the feature value is 1
        right_split = data[data[feature] == 1]


        # Compute the classification error of this split.
        if criteria == "error":
            # Calculate the number of misclassified examples in the left split.
            left_points = intermediate_node_num_mistakes(left_split[target])

            # Calculate the number of misclassified examples in the right split.

            right_points = intermediate_node_num_mistakes(right_split[target])

            # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points) 
            average_points = (left_points + right_points) / (num_data_points) 
            
        #Calulate left split and right split weight
        left_weight = len(left_split)/ num_data_points
        right_weight = len(right_split) / num_data_points

        #Compute the entropy 
        if criteria == "entropy": 
            #Calculate the criterion points in the left split
            left_points = calculateEntropy(left_split[target])

            #Calculate the criterion points in the right split
            right_points = calculateEntropy(right_split[target])

            #Calculate the average criterion
            average_points = (left_weight * left_points) + (right_weight *  right_points)

        #Compute the gini index 
        if criteria == "gini": 
            #Calculate the criterion points in the left split
            left_points = calculateGiniIndex(left_split[target])

            #Calculate the criterion points in the right split
            right_points = calculateGiniIndex(right_split[target])

            #Calculate the average criterion
            average_points = (left_weight * left_points) + (right_weight *  right_points)

        #update the best feature and point
        if average_points < best_percentage_points:
            best_feature = feature
            best_percentage_points = average_points

    return best_feature  # Return the best feature we found

To test your `best_splitting_feature` function, run the following code:

In [127]:
b1 = best_splitting_feature(train_data, features, target, 'error')
print(b1)

b2 = best_splitting_feature(train_data, features, target, 'entropy')
print(b2)

b3 = best_splitting_feature(train_data, features, target, 'gini')
print(b3)

term. 36 months
grade.A
grade.A


In [128]:
train_data.print_rows(num_rows=10)

+------------+---------+---------+---------+---------+---------+---------+---------+
| safe_loans | grade.A | grade.B | grade.C | grade.D | grade.E | grade.F | grade.G |
+------------+---------+---------+---------+---------+---------+---------+---------+
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    0    |    0    |    0    |    1    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    | 

In [129]:
if best_splitting_feature(train_data, features, target, 'error') == 'term. 36 months':
    print('Test passed!')
else:
    print('Test failed... try again!')

Test passed!


## Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'feature'            : The feature that this node splits on.
    }

In [130]:
class Node:

    def __init__(self):
        self.feature = None
        self.left = None
        self.right = None
        self.is_leaf = False
        self.prediction = None

    def __init__(self, feature=None, left=None, right=None, is_leaf=False, prediction=None):
        self.feature = feature
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.prediction = prediction

    def setPrediction(self, predict):
        self.prediction = predict


In [131]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = Node()
    leaf.is_leaf = True

    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf.prediction = 1  ## YOUR CODE HERE
    else:
        leaf.prediction = -1  ## YOUR CODE HERE

    # Return the leaf node        
    return leaf

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 

Now, we will write down the skeleton of the learning algorithm. Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

Here is a recursive function to count the nodes in your tree:

In [132]:
def decision_tree_create(
    data, features, target, current_depth=0, max_depth=10, criteria="error"
):
    remaining_features = features[:]  # Make a copy of the features.

    target_values = data[target]

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    if intermediate_node_num_mistakes(target_values) == 0:
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)

    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == []:
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)

        # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:
        # print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature = best_splitting_feature(
        data, features, target, criteria=criteria
    )

    # Split on the best feature that we found.
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]

    remaining_features.remove(splitting_feature)

    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        # print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        # print("Creating leaf node.")
        return create_leaf(right_split[target])

    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(
        left_split, remaining_features, target, current_depth + 1, max_depth, criteria
    )
    ## YOUR CODE HERE
    right_tree = decision_tree_create(
        right_split, remaining_features, target, current_depth + 1, max_depth, criteria
    )

    currentNode = Node(splitting_feature, left_tree, right_tree, False, None)

    return currentNode


In [133]:
def count_nodes(tree):
    if tree.is_leaf:
        return 1
    return 1 + count_nodes(tree.left) + count_nodes(tree.right)

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

## Build the tree!

Now that all the tests are passing, we will train a tree model on the **train_data**. Test 3 different trees use differenct criterias method: classfication error, entropy and gini.

**Warning**: This code block may take 1-2 minutes to learn. 

In [134]:
my_decision_tree = decision_tree_create(
    train_data, features, target, current_depth=0, max_depth=4
)

print("Done error decision tree")

Done error decision tree


In [135]:
my_decision_tree_entropy = decision_tree_create(
    train_data, features, target, current_depth=0, max_depth=4,criteria='entropy'
)

print("Done entropy decision tree")

Done entropy decision tree


In [136]:

my_decision_tree_gini = decision_tree_create(
    train_data, features, target, current_depth=0, max_depth=4,criteria='gini'
)

print("Done gini decision tree")

Done gini decision tree


## Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [137]:
def classify(tree, x, annotate=False):
    # if the node is a leaf node.
    if tree.is_leaf:
        if annotate:
            print("At leaf, predicting %s" % tree.prediction)
        return tree.prediction
    else:
        # split on feature.
        split_feature_value = x[tree.feature]

        if annotate:
            print("Split on %s = %s" % (tree.feature, split_feature_value))
        if split_feature_value == 0:
            return classify(tree.left, x, annotate)
        else:
            return classify(tree.right, x, annotate)

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [138]:
test_data[0]

{'safe_loans': -1,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'term. 36 months': 0,
 'term. 60 months': 1,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'short_emp.0': 1,
 'short_emp.1': 0,
 'is_inc_v.Not Verified': 1,
 'is_inc_v.Source Verified': 0,
 'is_inc_v.Verified': 0}

In [139]:
print('Predicted class: %s ' % classify(my_decision_tree, test_data[0]))

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [140]:
classify(my_decision_tree, test_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
At leaf, predicting -1


-1

## Evaluating your decision tree

Now, we will write a function to evaluate a decision tree by computing the accuracy of the tree on the given dataset.

In [141]:
def evaluate_accuracy(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))

    # Once you've made the predictions, calculate the classification error and return it
    trueprediction = 0
    for i, record in enumerate(data):
        if (record[target] == prediction[i]):
            trueprediction += 1

    return trueprediction / len(data) * 100

Now, let's use this function to evaluate the classification error on the test set.

In [142]:
print(evaluate_accuracy(my_decision_tree, test_data, target))

print(evaluate_accuracy(my_decision_tree_entropy, test_data, target))

print(evaluate_accuracy(my_decision_tree_gini, test_data, target))


61.169754416199915
61.83757001292546
61.83757001292546
